# Feature table calculation in action
## load data

In [8]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from spatial_tools.image._utils import *
from spatial_tools.image.manipulate import crop_img
from spatial_tools.image.tools import read_tif, get_image_features

import skimage.feature as sk_image

# path to "raw" dataset folder
BASE_PATH = "/storage/groups/ml01/datasets/raw/20200909_PublicVisium_giovanni.palla"
dataset_name = "V1_Adult_Mouse_Brain"
dataset_folder = os.path.join(
    BASE_PATH, "20191205_10XVisium_MouseBrainCoronal_giovanni.palla"
)

## example of calculating the feature table, currently just supporting hog features

In [9]:
# read the corresponding adata object
adata = sc.read_visium(
    dataset_folder, count_file=f"{dataset_name}_filtered_feature_bc_matrix.h5"
)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


## select which features to add - the function defaults to calculating summary statistics

### the available feature in the current version are: "hog", "texture", "summary", "color_hist"


In [10]:
features = features=["hog", "texture", "summary", "color_hist"]

### call the get_image_feature function, here pass dataset_folder and dataset_name for loading image

#### subset the adata object directly in the function argument to select the spot ids to calculate features forr

In [13]:
feature_table = get_image_features(adata[0:10], dataset_folder, dataset_name, features)
feature_table

/home/icb/olle.holmberg/anaconda3/envs/hackathon-spatial/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


,hog_0,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,hog_8,hog_9,...,color_hist_ch_2_bin_0,color_hist_ch_2_bin_1,color_hist_ch_2_bin_2,color_hist_ch_2_bin_3,color_hist_ch_2_bin_4,color_hist_ch_2_bin_5,color_hist_ch_2_bin_6,color_hist_ch_2_bin_7,color_hist_ch_2_bin_8,color_hist_ch_2_bin_9
cell_name,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,0.142508,0.055804,0.010893,0.038074,0.103839,0.100785,0.022411,0.099197,0.153254,0.076143,...,0,2,235,1976,4098,1359,238,11,2,0
AAACAATCTACTAGCA-1,0.196273,0.180013,0.153482,0.031666,0.115323,0.049121,0.052532,0.033917,0.049992,0.074650,...,0,5,380,2506,3688,1181,160,1,0,0
AAACACCAATAACTGC-1,0.085761,0.040085,0.015754,0.078683,0.145507,0.090550,0.040465,0.033820,0.022875,0.213458,...,0,1,178,871,5708,1162,1,0,0,0
AAACAGAGCGACTCCT-1,0.086159,0.023431,0.119787,0.073737,0.210138,0.060925,0.034919,0.060748,0.024580,0.099000,...,0,4,171,1926,4193,1451,170,6,0,0
AAACCGGGTAGGTACC-1,0.212877,0.072862,0.084954,0.113582,0.173368,0.032719,0.045506,0.026452,0.051656,0.098131,...,0,0,76,1777,4661,1365,42,0,0,0
AAACCGTTCGTCCAGG-1,0.150738,0.215759,0.149532,0.100038,0.187134,0.023664,0.086682,0.074917,0.038891,0.116227,...,0,32,591,2115,3051,1721,400,11,0,0
AAACCTCATGAAGTTG-1,0.174012,0.058884,0.066128,0.021685,0.095461,0.079950,0.104693,0.050519,0.019576,0.222078,...,0,3,130,1521,4903,1360,4,0,0,0
AAACGAAGAACATACC-1,0.140165,0.042695,0.023924,0.045290,0.216118,0.216118,0.216118,0.095493,0.027842,0.123821,...,0,31,354,2527,3587,1197,214,11,0,0
AAACGAGACGGTTGAT-1,0.210230,0.210230,0.210230,0.095114,0.158538,0.030380,0.020032,0.024770,0.045952,0.146233,...,0,229,617,1813,3022,1752,455,33,0,0
